In [1]:
%matplotlib notebook

import multiprocessing.pool
import pathlib
import time

import matplotlib.pyplot as plt
import torch
import tqdm

from laueimproc import Diagram
from laueimproc.io.download import get_samples  # gives access to the dataset
from laueimproc.opti.manager import DiagramManager

In [8]:
manager = DiagramManager()
manager.verbose = True  # to display the operations in background
manager.max_mem_percent = 75  # trigger cache clear after 75% of ram full, default based on swapiness
manager.update()  # dereference unreachable diagrams

update tracked diagrams, 99 are freed, 10001 remain


In [7]:
all_files = sorted(get_samples().iterdir())*100
diagrams = [Diagram(f) for f in all_files]
for diagram in diagrams:
    diagram.find_spots()

In [6]:
"""Test compression ROIS"""

from laueimproc.opti.comp import compress_rois, decompress_rois
from laueimproc.opti.cache import getsizeof

brut = comp = 0
t_start = time.time()
for diagram in tqdm.tqdm(diagrams):
    rois_comp = compress_rois(diagram.rois)
    brut += getsizeof(diagram.rois)
    comp += getsizeof(rois_comp)
    # print(f"avant={getsizeof(diagram.rois)}, apres={getsizeof(rois_comp)}")
    assert torch.max(torch.abs(diagram.rois - decompress_rois(rois_comp))).item() <= 1/131070
print((time.time()-t_start)/len(diagrams))
print(f"total {brut//1000} to {comp//1000} (ratio {brut/comp:.1f})")

100%|████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.26it/s]

0.09754601001739502
total 416919 to 3781 (ratio 110.2)


try to clear 56.7M bytes of cache
58.6M bytes of cache are removed (level 1)
try to clear 226.6M bytes of cache
226.7M bytes of cache are removed (level 1)
try to clear 207.3M bytes of cache
220.2M bytes of cache are removed (level 2)
try to clear 1.9G bytes of cache
1.4G bytes of cache are removed (level 2)
try to clear 582.3M bytes of cache
5.8K bytes of cache are removed (level 2)
try to clear 669.2M bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 729.3M bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 727.4M bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 785.2M bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 776.0M bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 775.5M bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 778.5M bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 707.3M bytes of cache
0.0B bytes of cache are removed (level 

try to clear 2.0G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.0G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.0G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.0G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.1G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.2G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.2G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.3G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.4G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.5G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.6G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.6G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.6G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.6G bytes o

try to clear 1.6G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 1.6G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 1.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 1.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.0G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.0G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.1G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.1G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.3G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.4G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.4G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.5G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.5G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.4G bytes o

try to clear 3.9G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 3.9G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.0G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.0G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.0G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.1G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.2G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.3G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.3G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.3G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.5G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.5G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.5G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 4.5G bytes o

try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes of cache
0.0B bytes of cache are removed (level 2)
try to clear 2.7G bytes o

In [ ]:
"""Test memory limit"""

for diagram in tqdm.tqdm(diagrams, smoothing=0.01):
    intensities = diagram.compute_pxl_intensities()
    sorted_indexs = torch.argsort(intensities, descending=True)
    diagram.filter_spots(sorted_indexs, msg="sorted by intensities", inplace=True)

  7%|███▎                                              | 668/10000 [00:38<08:35, 18.11it/s]

try to clear 34.6M bytes of cache
35.6M bytes of cache are removed (level 1)


  7%|███▋                                              | 741/10000 [00:43<08:54, 17.32it/s]

try to clear 12.1M bytes of cache


  8%|███▊                                              | 762/10000 [00:44<08:49, 17.45it/s]

...

 10%|█████                                            | 1021/10000 [01:58<23:07,  6.47it/s]

561.9M bytes of cache are removed (level 1)
try to clear 795.1M bytes of cache


 11%|█████▏                                           | 1065/10000 [01:58<21:25,  6.95it/s]

803.9M bytes of cache are removed (level 2)
try to clear 685.9M bytes of cache
701.2M bytes of cache are removed (level 2)
try to clear 230.3M bytes of cache


 11%|█████▎                                           | 1078/10000 [02:08<23:20,  6.37it/s]ERROR:root:'bytes' object does not support item assignment
